# Cardiopulmonary Events from Smoke Estimator (CENSE)

A web application for visualizing relative health risk for various medical conditions related to particulate matter (PM2.5).

```
AIRPACT5 forecases from Joe Vaughn, Washington State University 
Concentration Responses Functions from Matt Kadlec, WA Dept of Ecology
Code written by Von P. Walden, Washington State University
```

In [1]:
import ipywidgets as widgets
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.feature import NaturalEarthFeature
%matplotlib inline

In [2]:
from AIRPACT5_CENSE import relativeHealthRisk

In [3]:
#....Date
date   = pd.to_datetime('now') - pd.Timedelta('1D')
#....Location (Spokane, WA)
latitude  = 47.6588
longitude = -117.4260
# ....Medical condition
conditions = ['Non-trauma EDV CRFs [ICD 10: A00-R99]',
              'Asthma HA CRFs [ICD 10: J45]',
              'WA baseline incidence rate (per person per day)',
              'Asthma symptoms onset CRFs [ICD 10: J45]',
              'COPD Emergency HA  CRFs [ICD 10: J41,J42,J43,J44]',
              'Acute Otitis Media (ear ache) Outpatient healthcare CRFs [ICD-10: H65, H66, H67, H68, H69, J10, J11, J12]',
              'Respiratory Disease HA CRFs [ICD 10: J00-J99]',
              'Ischemic heart disease EDV CRFs [ICD 10: I20-I25, I46, I49]']

In [11]:
@widgets.interact(age=list(np.arange(0,111)), condition=conditions)
def f(age=25, condition=conditions[3]):
    personalRiskIncrease, riskIncrease, CRF = relativeHealthRisk(date, latitude, longitude, age, condition)
    grid = xr.open_dataset('GRIDCRO2D')
    lats = np.reshape(grid.LAT.values,(258,285))
    lons = np.reshape(grid.LON.values,(258,285))
    plt.figure(figsize=(16,8))
    ax = plt.axes(projection=ccrs.PlateCarree())
    states = NaturalEarthFeature(category="cultural", scale="50m", facecolor="none", name="admin_1_states_provinces_shp")
    ax.add_feature(states, linewidth=.5, edgecolor="black")
    plt.contourf(lons, lats, riskIncrease, np.arange(0,1.1,0.1), transform=ccrs.PlateCarree())
    ax.coastlines(resolution='10m')
    plt.colorbar()
    plt.title(condition + ' for ' + str(age) + ' year old')
    plt.show()
    
    plt.figure(figsize=(16,6))
    plt.plot(CRF.index, CRF)
    plt.plot(age,CRF[age],'ro', markersize=20)
    plt.xlabel('Age (years)');
    plt.ylabel('CRF')
    plt.title('Concentration Response Function for ' + condition + ', from Matt Kadlec, WA Dept of Ecology')
    plt.show()

interactive(children=(Dropdown(description='age', index=25, options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,…